In [29]:
import base64
txt = '_'.join([''.join([str(i) for i in range(10)])] * 10)
utf = txt.encode()
b64 = base64.b64encode(utf)

print(f"""{txt=}
{utf=}
{len(utf)=}
{b64=}
{len(b64)=}
""")

txt='0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789'
utf=b'0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789'
len(utf)=109
b64=b'MDEyMzQ1Njc4OV8wMTIzNDU2Nzg5XzAxMjM0NTY3ODlfMDEyMzQ1Njc4OV8wMTIzNDU2Nzg5XzAxMjM0NTY3ODlfMDEyMzQ1Njc4OV8wMTIzNDU2Nzg5XzAxMjM0NTY3ODlfMDEyMzQ1Njc4OQ=='
len(b64)=148



In [30]:
def repeated_key_xor(ptxt_bytes, key_bytes):
    ctxt_bytes = []
    key_i = 0
    for pbyte in ptxt_bytes:
        ctxt_bytes.append(pbyte ^ key_bytes[key_i])
        key_i = (key_i + 1) % len(key_bytes)

    out = ''
    for cb in ctxt_bytes:
        out += f'{cb:02x}'
    return out

In [33]:
key = 'HOPS'
ctxt = repeated_key_xor(utf, key.encode())
ctxt

'787e62607c7a666470760f63797d63677d79676b711060627a7c64667e78686a177f61617b7b65657f77690c787e62607c7a666470760f63797d63677d79676b711060627a7c64667e78686a177f61617b7b65657f77690c787e62607c7a666470760f63797d63677d79676b71'

### debugging

In [75]:
ptxt_bytes

b'0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789'

In [79]:
hex(ord('4') ^ ord('H'))

'0x7c'

### end debugging

In [34]:
ctxt_bytes = bytes.fromhex(ctxt)
ctxt_bytes

b'x~b`|zfdpv\x0fcy}cg}ygkq\x10`bz|df~xhj\x17\x7faa{{ee\x7fwi\x0cx~b`|zfdpv\x0fcy}cg}ygkq\x10`bz|df~xhj\x17\x7faa{{ee\x7fwi\x0cx~b`|zfdpv\x0fcy}cg}ygkq'

In [37]:
repeated_key_xor(ctxt_bytes, key.encode())

'303132333435363738395f303132333435363738395f303132333435363738395f303132333435363738395f303132333435363738395f303132333435363738395f303132333435363738395f303132333435363738395f303132333435363738395f30313233343536373839'

In [38]:
bytes.fromhex(repeated_key_xor(ctxt_bytes, key.encode())).decode()

'0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789'

In [39]:
ptxt_bytes = bytes.fromhex(repeated_key_xor(ctxt_bytes, key.encode()))
ptxt_bytes

b'0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789_0123456789'

In [53]:
def break_into_blocks(data, block_size):
    data_blocks = []
    for start in range(0,len(data),block_size):
        data_blocks.append(data[start:start+block_size])
    return data_blocks

def transpose_block(data_blocks, block_size):
    data_blocks_T = []
    for idx in range(block_size):
        block_T = []
        last_block = False
        for block in data_blocks:
            assert not last_block, f'only last block may have missing entries.  found non-final block with less than {block_size=} entries'
            if idx < len(block):
                block_T.append(block[idx])
            else:
                # should not be any more blocks after this one
                last_block=True
        data_blocks_T.append(block_T)
    return data_blocks_T

In [54]:
ctxt_blocks = break_into_blocks(ctxt_bytes, len(key))
ctxt_blocks

[b'x~b`',
 b'|zfd',
 b'pv\x0fc',
 b'y}cg',
 b'}ygk',
 b'q\x10`b',
 b'z|df',
 b'~xhj',
 b'\x17\x7faa',
 b'{{ee',
 b'\x7fwi\x0c',
 b'x~b`',
 b'|zfd',
 b'pv\x0fc',
 b'y}cg',
 b'}ygk',
 b'q\x10`b',
 b'z|df',
 b'~xhj',
 b'\x17\x7faa',
 b'{{ee',
 b'\x7fwi\x0c',
 b'x~b`',
 b'|zfd',
 b'pv\x0fc',
 b'y}cg',
 b'}ygk',
 b'q']

In [57]:
ctxt_blocks_T = transpose_block(ctxt_blocks, len(key))
[''.join([chr(i) for i in ls]) for ls in ctxt_blocks_T]


['x|py}qz~\x17{\x7fx|py}qz~\x17{\x7fx|py}q',
 '~zv}y\x10|x\x7f{w~zv}y\x10|x\x7f{w~zv}y',
 'bf\x0fcg`dhaeibf\x0fcg`dhaeibf\x0fcg',
 '`dcgkbfjae\x0c`dcgkbfjae\x0c`dcgk']

In [59]:
ctxt_blocks_T_hex = [''.join([f'{i:02x}' for i in ls]) for ls in ctxt_blocks_T]
ctxt_blocks_T_hex

['787c70797d717a7e177b7f787c70797d717a7e177b7f787c70797d71',
 '7e7a767d79107c787f7b777e7a767d79107c787f7b777e7a767d79',
 '62660f636760646861656962660f636760646861656962660f6367',
 '606463676b62666a61650c606463676b62666a61650c606463676b']

In [60]:
b = bytes.fromhex(ctxt_blocks_T_hex[0])
b

b'x|py}qz~\x17{\x7fx|py}qz~\x17{\x7fx|py}q'

In [64]:
ctxt_blocks_T[0]

[120,
 124,
 112,
 121,
 125,
 113,
 122,
 126,
 23,
 123,
 127,
 120,
 124,
 112,
 121,
 125,
 113,
 122,
 126,
 23,
 123,
 127,
 120,
 124,
 112,
 121,
 125,
 113]

In [92]:
[i^ord('H') for i in ctxt_blocks_T[0]]

[48,
 52,
 56,
 49,
 53,
 57,
 50,
 54,
 95,
 51,
 55,
 48,
 52,
 56,
 49,
 53,
 57,
 50,
 54,
 95,
 51,
 55,
 48,
 52,
 56,
 49,
 53,
 57]

In [95]:
int('H'.encode().hex(),16)

72

In [99]:
for block_i in range(len(key)):
    print(''.join([chr(i ^ ord(key[block_i])) for i in ctxt_blocks_T[block_i]]))

04815926_3704815926_37048159
15926_3704815926_3704815926
26_3704815926_3704815926_37
3704815926_3704815926_37048


In [74]:
hex( ord('0') ^ ord('H') ^ ord('H'))

'0x30'